# Objectives {.unnumbered}

By the end of this lab, you will:
1. Load and analyze the **Lightcast dataset** in **Spark DataFrame**.
2. Create **five easy and three medium-complexity visualizations** using **Plotly**.
3. Explore **salary distributions, employment trends, and job postings**.
4. Analyze **skills in relation to NAICS/SOC/ONET codes and salaries**.
5. Customize **colors, fonts, and styles** in all visualizations (**default themes result in a 2.5-point deduction**).
6. Follow **best practices for reporting on data communication**.

# Step 1: Load the Dataset {.unnumbered}


In [2]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "vscode"
from pyspark.sql import SparkSession
from pyspark.sql.functions import col


# Initialize Spark Session
spark = SparkSession.builder.appName("LightcastData").getOrCreate()

# Load Data
df = spark.read.option("header", "true").option("inferSchema", "true").option("multiLine","true").option("escape", "\"").csv("./data/lightcast_job_postings.csv")

# Show Schema and Sample Data
df.printSchema()
df.show(5)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/24 23:02:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


root
 |-- ID: string (nullable = true)
 |-- LAST_UPDATED_DATE: string (nullable = true)
 |-- LAST_UPDATED_TIMESTAMP: timestamp (nullable = true)
 |-- DUPLICATES: integer (nullable = true)
 |-- POSTED: string (nullable = true)
 |-- EXPIRED: string (nullable = true)
 |-- DURATION: integer (nullable = true)
 |-- SOURCE_TYPES: string (nullable = true)
 |-- SOURCES: string (nullable = true)
 |-- URL: string (nullable = true)
 |-- ACTIVE_URLS: string (nullable = true)
 |-- ACTIVE_SOURCES_INFO: string (nullable = true)
 |-- TITLE_RAW: string (nullable = true)
 |-- BODY: string (nullable = true)
 |-- MODELED_EXPIRED: string (nullable = true)
 |-- MODELED_DURATION: integer (nullable = true)
 |-- COMPANY: integer (nullable = true)
 |-- COMPANY_NAME: string (nullable = true)
 |-- COMPANY_RAW: string (nullable = true)
 |-- COMPANY_IS_STAFFING: boolean (nullable = true)
 |-- EDUCATION_LEVELS: string (nullable = true)
 |-- EDUCATION_LEVELS_NAME: string (nullable = true)
 |-- MIN_EDULEVELS: integer (

25/03/24 23:02:41 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+-----------------+----------------------+----------+--------+---------+--------+--------------------+--------------------+--------------------+-----------+-------------------+--------------------+--------------------+---------------+----------------+--------+--------------------+-----------+-------------------+----------------+---------------------+-------------+-------------------+-------------+------------------+---------------+--------------------+--------------------+--------------------+-------------+------+-----------+----------------+-------------------+---------+-----------+--------------------+--------------------+-------------+------+--------------+-----+--------------------+-----+----------+---------------+--------------------+---------------+--------------------+------------+--------------------+------------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+--------------------+------+------

# Salary Distribution by Employment Type
- Identify salary trends across different employment types.
- **Filter the dataset**
  - Remove records where **salary is missing or zero**.
- **Aggregate Data**
  - Group by **employment type** and compute salary distribution.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `EMPLOYMENT_TYPE_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize **colors, fonts, and styles** to avoid a **2.5-point deduction**.
- **Explanation:** Write two sentences about what the graph reveals.



In [ ]:
# Your Code for 1st question here
from pyspark.sql.functions import col

df_filtered = df.filter((col("SALARY_FROM").isNotNull()) & (col("SALARY_FROM") > 0))

pdf = df_filtered.select("EMPLOYMENT_TYPE_NAME", "SALARY_FROM").toPandas()

import plotly.express as px

fig = px.box(
    pdf,
    x="EMPLOYMENT_TYPE_NAME",
    y="SALARY_FROM",
    title="Salary Distribution by Employment Type",
    color_discrete_sequence=[
        "#636EFA", "#EF553B", "#00CC96", "#AB63FA", "#FFA15A", "#19D3F3"
    ]  
)


fig.update_layout(
    font_family="Courier New",         
    font_size=30,
    title_font=dict(size=20, family="Georgia", color="#333333"),
    plot_bgcolor="#F9F9F9",            
    paper_bgcolor="#FFFFFF"            
)

fig.show()

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


# 1
The box plot shows that full-time positions tend to offer higher starting salaries compared to part-time roles. However, salary variability is also greater for full-time roles, indicating a wider range of compensation levels within that category.

# Salary Distribution by Industry
- Compare salary variations across industries.
- **Filter the dataset**
  - Keep records where **salary is greater than zero**.
- **Aggregate Data**
  - Group by **NAICS industry codes**.
- **Visualize results**
  - Create a **box plot** where:
    - **X-axis** = `NAICS2_NAME`
    - **Y-axis** = `SALARY_FROM`
  - Customize colors, fonts, and styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 2nd question here
import plotly.express as px

df_filtered = df.filter(col("SALARY_FROM") > 0)

pdf = df_filtered.select("NAICS2_NAME", "SALARY_FROM").toPandas()

fig = px.box(
    pdf,
    x="NAICS2_NAME",
    y="SALARY_FROM",
    title="Salary Distribution by Industry (NAICS2)",
    color_discrete_sequence=["#FF6B6B", "#FFD93D", "#6BCB77", "#4D96FF", "#9D4EDD"]
)

fig.update_layout(
    width=1500,
    xaxis_tickangle=-10,
    font_family="Verdana",
    font_size=14,
    title_font=dict(size=20, family="Georgia", color="#333333"),
    plot_bgcolor="#F0F0F0",
    paper_bgcolor="#FFFFFF"
)

fig.show()

# 2
The box plot reveals that salary distributions vary significantly across industries. Some sectors, such as "Administrative and Support and Waste Management and Remediation Services" and "Unclassified Industry", show higher median salaries and wider variability, indicating a range of compensation levels. Others, like "Finance and Insurance" and "Manufacturing", tend to offer more consistent but lower salary levels.

# Job Posting Trends Over Time
- Analyze how job postings fluctuate over time.
- **Aggregate Data**
  - Count job postings per **posted date (`POSTED`)**.
- **Visualize results**
  - Create a **line chart** where:
    - **X-axis** = `POSTED`
    - **Y-axis** = `Number of Job Postings`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [44]:
# Your code for 3rd question here
from pyspark.sql.functions import col

df_trend = df.filter(col("POSTED").isNotNull())
df_trend = df_trend.withColumn("POSTED_MONTH", col("POSTED").substr(1, 7))

df_monthly = df_trend.groupBy("POSTED_MONTH").count().orderBy("POSTED_MONTH")
pdf_monthly = df_monthly.toPandas()

import plotly.express as px

fig = px.line(
    pdf_monthly,
    x="POSTED_MONTH",
    y="count",
    title="Job Posting Trends Over Time",
    markers=True,
    line_shape="spline"
)

fig.update_layout(
    xaxis_tickangle=-45,
    font_family="Tahoma",
    font_size=14,
    title_font=dict(size=22, family="Georgia", color="#222222"),
    plot_bgcolor="#F9F9F9",
    paper_bgcolor="#FFFFFF"
)

fig.show()

# 3  
The line chart reveals strong monthly seasonality in job postings, with noticeable peaks and drops in a consistent pattern.
These patterns suggest cyclical hiring trends, potentially tied to fiscal periods or academic calendars.

# Top 10 Job Titles by Count
- Identify the most frequently posted job titles.
- **Aggregate Data**
  - Count the occurrences of each **job title (`TITLE_NAME`)**.
  - Select the **top 10 most frequent titles**.
- **Visualize results**
  - Create a **bar chart** where:
    - **X-axis** = `TITLE_NAME`
    - **Y-axis** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [22]:
# Your code for 4th question here
df_top = df.groupBy("TITLE_RAW").count().orderBy("count", ascending=False).limit(10)
pdf_top = df_top.toPandas()

import plotly.express as px

fig = px.bar(
    pdf_top,
    x="TITLE_RAW",
    y="count",
    title="Top 10 Job Titles by Count",
    color="count",
    color_continuous_scale="Blues"
)

fig.update_layout(
    font_family="Arial",
    font_size=14,
    title_font=dict(size=20, family="Georgia", color="#222222"),
    plot_bgcolor="#f9f9f9",
    paper_bgcolor="#ffffff",
    xaxis_tickangle=-10
)

fig.show()


# 4
The bar chart shows that "Data Analyst" is by far the most frequently posted job title, indicating strong demand for this role across industries. Other common titles such as "Enterprise Architect"

# Remote vs On-Site Job Postings
- Compare the proportion of remote and on-site job postings.
- **Aggregate Data**
  - Count job postings by **remote type (`REMOTE_TYPE_NAME`)**.
- **Visualize results**
  - Create a **pie chart** where:
    - **Labels** = `REMOTE_TYPE_NAME`
    - **Values** = `Job Count`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 5th question here
from pyspark.sql.functions import col

df_remote = df.filter(col("REMOTE_TYPE_NAME").isNotNull())

df_remote_count = df_remote.groupBy("REMOTE_TYPE_NAME").count()

pdf_remote = df_remote_count.toPandas()

import plotly.express as px

fig = px.pie(
    pdf_remote,
    names="REMOTE_TYPE_NAME",
    values="count",
    title="Remote vs On-Site Job Postings",
    color_discrete_sequence=px.colors.sequential.RdBu
)

fig.update_layout(
    font_family="Verdana",
    font_size=14,
    title_font=dict(size=22, family="Georgia", color="#222222"),
    plot_bgcolor="#F9F9F9",
    paper_bgcolor="#FFFFFF"
)

fig.show()

# 5
The pie chart shows that the majority of job postings did not specify a remote type, making up over 75% of the total. Among those that did, remote jobs significantly outnumber hybrid and on-site roles.

# Skill Demand Analysis by Industry (Stacked Bar Chart)
- Identify which skills are most in demand in various industries.
- **Aggregate Data**
  - Extract **skills** from job postings.
  - Count occurrences of skills grouped by **NAICS industry codes**.
- **Visualize results**
  - Create a **stacked bar chart** where:
    - **X-axis** = `Industry`
    - **Y-axis** = `Skill Count`
    - **Color** = `Skill`
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [3]:
# Your code for 6th question here
from pyspark.sql.functions import explode, from_json
from pyspark.sql.types import ArrayType, StringType

# 解析 JSON 列为数组
df_parsed = df.withColumn("SKILLS_ARRAY", from_json("SKILLS_NAME", ArrayType(StringType())))

# 展开技能列
df_skills = df_parsed.select("NAICS2_NAME", explode("SKILLS_ARRAY").alias("SKILL"))

# 统计各技能在各行业出现的次数
df_skill_counts = df_skills.groupBy("NAICS2_NAME", "SKILL").count()

# 取 TOP 15 技能（按出现频次）
df_top_skills = df_skill_counts.orderBy("count", ascending=False).limit(15)

# 转为 Pandas
pdf_skills = df_top_skills.toPandas()

# 可视化
import plotly.express as px

fig = px.bar(
    pdf_skills,
    x="NAICS2_NAME",
    y="count",
    color="SKILL",
    title="Top 15 Skill Demand by Industry",
    text_auto=True
)

fig.update_layout(
    barmode="stack",
    font_family="Verdana",
    font_size=12,
    title_font=dict(size=20, family="Georgia", color="#222222"),
    plot_bgcolor="#F9F9F9",
    paper_bgcolor="#FFFFFF",
    xaxis_tickangle=30
)

fig.show()


# 6
The chart shows that the highest skill demand is in the Professional, Scientific, and Technical Services industry. Skills like Communication, Management, and Data Analysis are consistently in high demand across multiple industries.


# Salary Analysis by ONET Occupation Type (Bubble Chart)
- Analyze how salaries differ across ONET occupation types.
- **Aggregate Data**
  - Compute **median salary** for each occupation in the **ONET taxonomy**.
- **Visualize results**
  - Create a **bubble chart** where:
    - **X-axis** = `ONET_NAME`
    - **Y-axis** = `Median Salary`
    - **Size** = Number of job postings
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 7th question here
from pyspark.sql.functions import col, avg, count

df_filtered = df.filter((col("SALARY_FROM").isNotNull()) & (col("SALARY_FROM") > 0))

df_onet_salary = df_filtered.groupBy("ONET_NAME").agg(
    avg("SALARY_FROM").alias("median_salary"),
    count("*").alias("job_postings")
)

pdf_onet = df_onet_salary.toPandas()
pdf_onet_sorted = pdf_onet.sort_values("median_salary", ascending=False)

import plotly.express as px

fig = px.scatter(
    pdf_onet_sorted,
    x="ONET_NAME",
    y="median_salary",
    size="job_postings",
    color="median_salary",
    color_continuous_scale="Viridis",
    title="Median Salary by ONET Occupation Type"
)

fig.update_layout(
    font_family="Verdana",
    font_size=12,
    title_font=dict(size=22, family="Georgia", color="#222222"),
    plot_bgcolor="#F9F9F9",
    paper_bgcolor="#FFFFFF",
    xaxis_tickangle=45
)

fig.show()

# 7
The data shows that only one ONET occupation type has salary data available, specifically for Business Intelligence Analysts.
This limits deeper comparative analysis, but it highlights the need for more comprehensive salary reporting across occupations.

# Career Pathway Trends (Sankey Diagram)
- Visualize job transitions between different occupation levels.
- **Aggregate Data**
  - Identify career transitions between **SOC job classifications**.
- **Visualize results**
  - Create a **Sankey diagram** where:
    - **Source** = `SOC_2021_2_NAME`
    - **Target** = `SOC_2021_3_NAME`
    - **Value** = Number of transitions
  - Apply custom colors and font styles.
- **Explanation:** Write two sentences about what the graph reveals.

In [ ]:
# Your code for 8th question here
df_pathways = df.select("SOC_2021_2_NAME", "SOC_2021_3_NAME") \
                .filter((col("SOC_2021_2_NAME").isNotNull()) & (col("SOC_2021_3_NAME").isNotNull()))

df_grouped = df_pathways.groupBy("SOC_2021_2_NAME", "SOC_2021_3_NAME").count().toPandas()

all_labels = pd.unique(df_grouped[["SOC_2021_2_NAME", "SOC_2021_3_NAME"]].values.ravel("K")).tolist()
df_grouped["source"] = df_grouped["SOC_2021_2_NAME"].apply(lambda x: all_labels.index(x))
df_grouped["target"] = df_grouped["SOC_2021_3_NAME"].apply(lambda x: all_labels.index(x))

import plotly.graph_objects as go

fig = go.Figure(data=[go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        line=dict(color="black", width=0.5),
        label=all_labels
    ),
    link=dict(
        source=df_grouped["source"],
        target=df_grouped["target"],
        value=df_grouped["count"]
    ))])

fig.update_layout(
    title_text="Career Pathway Trends",
    font=dict(size=14, family="Arial"),
    plot_bgcolor="#F9F9F9",
    paper_bgcolor="#FFFFFF"
)

fig.show()

# 8
The Sankey diagram shows a clear transition pattern from "Computer and Mathematical Occupations" to "Mathematical Science Occupations."
This suggests a strong and direct career pathway within analytical or data-focused roles.